In [1]:
import pandas as pd
import numpy as np
import sklearn
import re
from sklearn import linear_model
import statsmodels.api as sm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import statistics 


## For 1m lag:

In [2]:
df_crsp = pd.read_csv("/Users/vlad_cherevkov/Desktop/ADV_inv_proj/crsp_final.csv")
df_rf = pd.read_csv("/Users/vlad_cherevkov/Desktop/ADV_inv_proj/risk_free_rate.csv")
df_rf.columns = ['date', 'rf']
df = pd.merge(df_crsp, df_rf, on='date', how='left')
df['RET-rf'] = df['RET'] - df['rf']
df['VOL-1m'] = df['VOL'].shift(-1)

In [3]:
g = df.groupby('date')
df_group = g.mean()

In [12]:
df_group

,PERMNO,PRC,VOL,RET,SHROUT,rf,RET-rf,VOL-1m,VOL-3m,VOL-6m,VOL-12m
date,,,,,,,,,,,
20060731,40878.322581,44.430000,1.888128e+06,0.004568,1.738667e+06,0.0040,0.000568,2.028497e+06,2.164632e+06,2.205955e+06,2.571033e+06
20060831,40878.322581,44.464446,2.028497e+06,0.022440,1.747538e+06,0.0042,0.018240,1.938866e+06,2.103691e+06,1.913183e+06,3.076586e+06
20060929,40424.597826,46.311957,1.959852e+06,0.030607,1.764230e+06,0.0041,0.026507,2.188058e+06,1.945852e+06,2.433049e+06,2.123057e+06
20061031,40424.597826,47.984673,2.188058e+06,0.047924,1.760963e+06,0.0041,0.043824,2.126494e+06,2.229917e+06,2.073615e+06,2.625664e+06
20061130,40424.597826,48.881374,2.126494e+06,0.021876,1.756208e+06,0.0042,0.017676,1.945852e+06,1.933934e+06,2.283878e+06,3.073488e+06
20061229,40031.780220,50.443111,1.961475e+06,0.015310,1.738978e+06,0.0040,0.011310,2.244073e+06,2.452765e+06,2.370704e+06,2.265799e+06
20070131,39550.666667,50.748389,2.265812e+06,0.021722,1.763980e+06,0.0044,0.017322,1.966267e+06,2.108767e+06,2.646729e+06,3.894311e+06
20070228,39550.666667,50.584333,1.966267e+06,-0.011680,1.766246e+06,0.0038,-0.015480,2.475973e+06,2.325503e+06,3.164163e+06,3.001316e+06
20070330,39550.666667,50.928333,2.475973e+06,0.008142,1.760762e+06,0.0043,0.003842,2.108767e+06,2.394502e+06,2.157797e+06,3.319068e+06


In [4]:
X = list(df_group['VOL-1m'])
Y = list(df_group['RET-rf'])
model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.428
Date:                Sat, 18 May 2019   Prob (F-statistic):              0.235
Time:                        18:56:31   Log-Likelihood:                 179.56
No. Observations:                 120   AIC:                            -357.1
Df Residuals:                     119   BIC:                            -354.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.712e-09   1.43e-09      1.195      0.235   -1.13e-09    4.55e-09
==============================================================================
Omnibus:                       18.693   Durbin-Watson:                   1.546
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.285
Skew:                          -0.522   Prob(JB):                     3.27e-11
Kurtosis:                       5.927   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
model.params

array([1.71182827e-09])

In [5]:
pred = X*model.params
significance = statistics.stdev(pred)/statistics.mean(Y)
print("Significance is: ", significance)

Significance is:  0.2921894249325511


## For other lags:

In [6]:
#3m
df['VOL-3m'] = df['VOL'].shift(-3)
g = df.groupby('date')
df_group = g.mean()

X_3 = list(df_group['VOL-3m'])
Y_3 = list(df_group['RET-rf'])
model_3 = sm.OLS(Y_3, X_3).fit()
model_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.449
Date:                Sat, 18 May 2019   Prob (F-statistic):              0.231
Time:                        18:56:33   Log-Likelihood:                 179.57
No. Observations:                 120   AIC:                            -357.1
Df Residuals:                     119   BIC:                            -354.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.726e-09   1.43e-09      1.204      0.231   -1.11e-09    4.56e-09
==============================================================================
Omnibus:                       17.817   Durbin-Watson:                   1.539
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.879
Skew:                          -0.479   Prob(JB):                     6.61e-11
Kurtosis:                       5.908   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
pred_3 = X_3*model_3.params
significance_3 = statistics.stdev(pred_3)/statistics.mean(Y_3)
print("Significance is: ", significance_3)

Significance is:  0.29430938407613083


In [8]:
#6m
df['VOL-6m'] = df['VOL'].shift(-6)
g = df.groupby('date')
df_group = g.mean()

X_6 = list(df_group['VOL-6m'])
Y_6 = list(df_group['RET-rf'])
model_6 = sm.OLS(Y_6, X_6).fit()
model_6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.5363
Date:                Sat, 18 May 2019   Prob (F-statistic):              0.465
Time:                        18:56:34   Log-Likelihood:                 179.11
No. Observations:                 120   AIC:                            -356.2
Df Residuals:                     119   BIC:                            -353.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.055e-09   1.44e-09      0.732      0.465    -1.8e-09    3.91e-09
==============================================================================
Omnibus:                       17.479   Durbin-Watson:                   1.531
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.230
Skew:                          -0.453   Prob(JB):                     5.55e-11
Kurtosis:                       5.937   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
pred_6 = X_6*model_6.params
significance_6 = statistics.stdev(pred_6)/statistics.mean(Y_6)
print("Significance is: ", significance_6)

Significance is:  0.1799540293544025


In [10]:
#12m
df['VOL-12m'] = df['VOL'].shift(-12)
g = df.groupby('date')
df_group = g.mean()

X_12 = list(df_group['VOL-12m'])
Y_12 = list(df_group['RET-rf'])
model_12 = sm.OLS(Y_12, X_12).fit()
model_12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.657
Date:                Sat, 18 May 2019   Prob (F-statistic):              0.200
Time:                        18:56:34   Log-Likelihood:                 179.67
No. Observations:                 120   AIC:                            -357.3
Df Residuals:                     119   BIC:                            -354.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.848e-09   1.44e-09      1.287      0.200   -9.95e-10    4.69e-09
==============================================================================
Omnibus:                       17.212   Durbin-Watson:                   1.548
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.432
Skew:                          -0.463   Prob(JB):                     2.25e-10
Kurtosis:                       5.834   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
pred_12 = X_6*model_12.params
significance_12 = statistics.stdev(pred_12)/statistics.mean(Y_12)
print("Significance is: ", significance_12)

Significance is:  0.315404105165743
